In [1]:
import pandas as pd 

In [2]:
data_path = '..\\data\\'

Чтение данных из CSV файлов

In [3]:
orders = pd.read_csv(data_path + 'orders.csv' )
order_products_train = pd.read_csv(data_path + 'order_products__train.csv')
order_products_prior = pd.read_csv(data_path + 'order_products__prior.csv')
products = pd.read_csv(data_path + 'products.csv')
aisles = pd.read_csv(data_path + 'aisles.csv')
departments = pd.read_csv(data_path + 'departments.csv')

Перевод полей типа "Object" в категориальный тип

In [4]:
aisles['aisle'] = aisles['aisle'].astype('category')
departments['department'] = departments['department'].astype('category')
orders['eval_set'] = orders['eval_set'].astype('category')
products['product_name'] = products['product_name'].astype('category')

Создание объекта DataFrame с заказами и соответствующими продуктами из prior части набора данных для генерации признаков

In [5]:
proir_orders_df = orders.merge(order_products_prior, on='order_id', how='inner')
proir_orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


### Подготовка признаков / Feature engineering

#### Признаки, относящиеся к пользователям

Размер последнего заказа пользователя

In [6]:
proir_orders_df.groupby(['user_id'])['order_number'].max().reset_index()

,user_id,order_number
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4
...,...,...
206204,206205,3
206205,206206,67
206206,206207,16
206207,206208,49


In [7]:
last_orders = proir_orders_df.merge(proir_orders_df.groupby(['user_id'])['order_number'].max().reset_index(), how='inner', left_on=['user_id', 'order_number'], right_on=['user_id', 'order_number'])

In [8]:
last_orders_df = last_orders.groupby(['user_id']).size().to_frame('last_order_size').reset_index()

Число заказов у пользователей

In [9]:
users_info = proir_orders_df.groupby('user_id')['order_number'].max().to_frame('u_total_orders').reset_index()
users_info.head()

,user_id,u_total_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


Как часто пользователь перезаказывал продукты

In [10]:
user_reorder_ratio = proir_orders_df.groupby('user_id')['reordered'].mean().to_frame('u_reordered_ratio').reset_index()
user_reorder_ratio.head()

,user_id,u_reordered_ratio
0,1,0.694915
1,2,0.476923
2,3,0.625000
3,4,0.055556
4,5,0.378378


In [11]:
users_info = users_info.merge(user_reorder_ratio, on='user_id', how='left')

In [12]:
users_info.head()

,user_id,u_total_orders,u_reordered_ratio
0,1,10,0.694915
1,2,14,0.476923
2,3,12,0.625000
3,4,5,0.055556
4,5,4,0.378378


In [13]:
users_info = users_info.merge(last_orders_df, on='user_id', how='left')

In [14]:
users_info.head()

,user_id,u_total_orders,u_reordered_ratio,last_order_size
0,1,10,0.694915,9
1,2,14,0.476923,16
2,3,12,0.625000,6
3,4,5,0.055556,3
4,5,4,0.378378,12


#### Признаки относящиеся к продуктам

Число заказов каждого продукта

In [15]:
items_info = proir_orders_df.groupby('product_id')['order_id'].count().to_frame('items_total_purchases').reset_index()
items_info.head()

,product_id,items_total_purchases
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


Вероятность продукта быть заказанным повторно

In [16]:
min_items_threshold = 40
items_reorder = proir_orders_df.groupby('product_id').filter(lambda x: x.shape[0] > min_items_threshold)
items_reorder.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [17]:
items_reorder = items_reorder.groupby('product_id')['reordered'].mean().to_frame('items_reorder_ratio').reset_index()
items_reorder.head()

,product_id,items_reorder_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,8,0.503030


Объединим признаки

In [18]:
items_info = items_info.merge(items_reorder, on='product_id', how='left')

In [19]:
items_info['items_reorder_ratio'] = items_info['items_reorder_ratio'].fillna(value=0)
items_info.head()

,product_id,items_total_purchases,items_reorder_ratio
0,1,1852,0.613391
1,2,90,0.133333
2,3,277,0.732852
3,4,329,0.446809
4,5,15,0.000000


Признаки пары пользователь-продукт

Как часто пользователь заказывал конкретный продукт

In [20]:
items_per_users = proir_orders_df.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('ipu_total_bought').reset_index()
items_per_users.head()

,user_id,product_id,ipu_total_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


Как часто пользователь заказывал продукт после своего первого заказа

Для этого поделим число заказов с покупкой этого продукта на число заказов с того момента, как пользователь купил продукт первый раз

In [21]:
times = proir_orders_df.groupby(['user_id', 'product_id'])[['order_id']].count()
times.columns = ['times_bought_n']
times.head()

Times_Bought_N
user_id product_id                
1       196                     10
        10258                    9
        10326                    1
        12427                   10
        13032                    3

In [22]:
total_orders = proir_orders_df.groupby('user_id')['order_number'].max().to_frame('total_orders')
total_orders.head()

,total_orders
user_id,
1,10
2,14
3,12
4,5
5,4


In [23]:
first_order_no = proir_orders_df.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number')
first_order_no  = first_order_no.reset_index()
first_order_no.head()

,user_id,product_id,first_order_number
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [24]:
span = pd.merge(total_orders, first_order_no, on='user_id', how='right')
span.head()

,user_id,total_orders,product_id,first_order_number
0,1,10,196,1
1,1,10,10258,2
2,1,10,10326,5
3,1,10,12427,1
4,1,10,13032,2


In [25]:
span['order_range_d'] = span.total_orders - span.first_order_number + 1
span.head()

,user_id,total_orders,product_id,first_order_number,Order_Range_D
0,1,10,196,1,10
1,1,10,10258,2,9
2,1,10,10326,5,6
3,1,10,12427,1,10
4,1,10,13032,2,9


In [26]:
uxp_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
uxp_ratio.head()

,user_id,product_id,Times_Bought_N,total_orders,first_order_number,Order_Range_D
0,1,196,10,10,1,10
1,1,10258,9,10,2,9
2,1,10326,1,10,5,6
3,1,12427,10,10,1,10
4,1,13032,3,10,2,9


In [27]:
uxp_ratio['uxp_reorder_ratio'] = uxp_ratio['times_bought_n'] / uxp_ratio['order_range_d]
uxp_ratio.head()

,user_id,product_id,Times_Bought_N,total_orders,first_order_number,Order_Range_D,uxp_reorder_ratio
0,1,196,10,10,1,10,1.000000
1,1,10258,9,10,2,9,1.000000
2,1,10326,1,10,5,6,0.166667
3,1,12427,10,10,1,10,1.000000
4,1,13032,3,10,2,9,0.333333


In [28]:
uxp_ratio = uxp_ratio.drop(['times_bought_n', 'total_orders', 'first_order_number', 'order_range_d'], axis=1)
uxp_ratio.head()

,user_id,product_id,uxp_reorder_ratio
0,1,196,1.000000
1,1,10258,1.000000
2,1,10326,0.166667
3,1,12427,1.000000
4,1,13032,0.333333


In [29]:
items_per_users = items_per_users.merge(uxp_ratio, on=['user_id', 'product_id'], how='left')
items_per_users.head()

,user_id,product_id,ipu_total_bought,uxp_reorder_ratio
0,1,196,10,1.000000
1,1,10258,9,1.000000
2,1,10326,1,0.166667
3,1,12427,10,1.000000
4,1,13032,3,0.333333


Сколько раз пользователь купил продукт за последние пять заказов

In [30]:
proir_orders_df['order_number_back'] = proir_orders_df.groupby('user_id')['order_number'].transform(max) - proir_orders_df.order_number +1 
proir_orders_df.head(15)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
0,2539329,1,prior,1,2,8,NaN,196,1,0,10
1,2539329,1,prior,1,2,8,NaN,14084,2,0,10
2,2539329,1,prior,1,2,8,NaN,12427,3,0,10
3,2539329,1,prior,1,2,8,NaN,26088,4,0,10
4,2539329,1,prior,1,2,8,NaN,26405,5,0,10
5,2398795,1,prior,2,3,7,15.0,196,1,1,9
6,2398795,1,prior,2,3,7,15.0,10258,2,0,9
7,2398795,1,prior,2,3,7,15.0,12427,3,1,9
8,2398795,1,prior,2,3,7,15.0,13176,4,0,9
9,2398795,1,prior,2,3,7,15.0,26088,5,1,9


In [31]:
five_proir_orders_df = proir_orders_df[proir_orders_df.order_number_back <= 5]
five_proir_orders_df.head(15)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
29,3367565,1,prior,6,2,7,19.0,196,1,1,5
30,3367565,1,prior,6,2,7,19.0,12427,2,1,5
31,3367565,1,prior,6,2,7,19.0,10258,3,1,5
32,3367565,1,prior,6,2,7,19.0,25133,4,1,5
33,550135,1,prior,7,1,9,20.0,196,1,1,4
34,550135,1,prior,7,1,9,20.0,10258,2,1,4
35,550135,1,prior,7,1,9,20.0,12427,3,1,4
36,550135,1,prior,7,1,9,20.0,25133,4,1,4
37,550135,1,prior,7,1,9,20.0,13032,5,1,4
38,3108588,1,prior,8,1,14,14.0,12427,1,1,3


In [32]:
last_five = five_proir_orders_df.groupby(['user_id','product_id'])[['order_id']].count()
last_five.columns = ['times_last5']
last_five.head(10)

times_last5
user_id product_id             
1       196                   5
        10258                 5
        12427                 5
        13032                 2
        25133                 5
        35951                 1
        38928                 1
        39657                 1
        46149                 3
        49235                 2

In [33]:
items_per_users = items_per_users.merge(last_five, on=['user_id', 'product_id'], how='left')

In [34]:
items_per_users = items_per_users.fillna(0)

Объединим все признаки

In [35]:
data_df = items_per_users.merge(users_info, on='user_id', how='left')
data_df.head()

,user_id,product_id,ipu_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,last_order_size
0,1,196,10,1.000000,5.0,10,0.694915,9
1,1,10258,9,1.000000,5.0,10,0.694915,9
2,1,10326,1,0.166667,0.0,10,0.694915,9
3,1,12427,10,1.000000,5.0,10,0.694915,9
4,1,13032,3,0.333333,2.0,10,0.694915,9


In [36]:
data_df = data_df.merge(items_info, on='product_id', how='left')
data_df.head()

,user_id,product_id,ipu_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,last_order_size,items_total_purchases,items_reorder_ratio
0,1,196,10,1.000000,5.0,10,0.694915,9,35791,0.776480
1,1,10258,9,1.000000,5.0,10,0.694915,9,1946,0.713772
2,1,10326,1,0.166667,0.0,10,0.694915,9,5526,0.652009
3,1,12427,10,1.000000,5.0,10,0.694915,9,6476,0.740735
4,1,13032,3,0.333333,2.0,10,0.694915,9,3751,0.657158


In [37]:
orders_future = orders[((orders.eval_set=='train') | (orders.eval_set=='test'))]
orders_future = orders_future[ ['user_id', 'eval_set', 'order_id'] ]

In [38]:
data_df = data_df.merge(orders_future, on='user_id', how='left')
data_df.head(10)

,user_id,product_id,ipu_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,last_order_size,items_total_purchases,items_reorder_ratio,eval_set,order_id
0,1,196,10,1.000000,5.0,10,0.694915,9,35791,0.776480,train,1187899
1,1,10258,9,1.000000,5.0,10,0.694915,9,1946,0.713772,train,1187899
2,1,10326,1,0.166667,0.0,10,0.694915,9,5526,0.652009,train,1187899
3,1,12427,10,1.000000,5.0,10,0.694915,9,6476,0.740735,train,1187899
4,1,13032,3,0.333333,2.0,10,0.694915,9,3751,0.657158,train,1187899
5,1,13176,2,0.222222,0.0,10,0.694915,9,379450,0.832555,train,1187899
6,1,14084,1,0.100000,0.0,10,0.694915,9,15935,0.810982,train,1187899
7,1,17122,1,0.166667,0.0,10,0.694915,9,13880,0.675576,train,1187899
8,1,25133,8,1.000000,5.0,10,0.694915,9,6196,0.740155,train,1187899
9,1,26088,2,0.200000,0.0,10,0.694915,9,2523,0.539041,train,1187899


In [39]:
data_train = data_df[data_df['eval_set'] == 'train']
data_train.head()

,user_id,product_id,ipu_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,last_order_size,items_total_purchases,items_reorder_ratio,eval_set,order_id
0,1,196,10,1.000000,5.0,10,0.694915,9,35791,0.776480,train,1187899
1,1,10258,9,1.000000,5.0,10,0.694915,9,1946,0.713772,train,1187899
2,1,10326,1,0.166667,0.0,10,0.694915,9,5526,0.652009,train,1187899
3,1,12427,10,1.000000,5.0,10,0.694915,9,6476,0.740735,train,1187899
4,1,13032,3,0.333333,2.0,10,0.694915,9,3751,0.657158,train,1187899


In [40]:
data_train = data_train.merge(order_products_train[['product_id','order_id', 'reordered']], on=['product_id','order_id'], how='left' )
data_train.head(15)

,user_id,product_id,ipu_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,last_order_size,items_total_purchases,items_reorder_ratio,eval_set,order_id,reordered
0,1,196,10,1.000000,5.0,10,0.694915,9,35791,0.776480,train,1187899,1.0
1,1,10258,9,1.000000,5.0,10,0.694915,9,1946,0.713772,train,1187899,1.0
2,1,10326,1,0.166667,0.0,10,0.694915,9,5526,0.652009,train,1187899,NaN
3,1,12427,10,1.000000,5.0,10,0.694915,9,6476,0.740735,train,1187899,NaN
4,1,13032,3,0.333333,2.0,10,0.694915,9,3751,0.657158,train,1187899,1.0
5,1,13176,2,0.222222,0.0,10,0.694915,9,379450,0.832555,train,1187899,NaN
6,1,14084,1,0.100000,0.0,10,0.694915,9,15935,0.810982,train,1187899,NaN
7,1,17122,1,0.166667,0.0,10,0.694915,9,13880,0.675576,train,1187899,NaN
8,1,25133,8,1.000000,5.0,10,0.694915,9,6196,0.740155,train,1187899,1.0
9,1,26088,2,0.200000,0.0,10,0.694915,9,2523,0.539041,train,1187899,1.0


In [41]:
data_train['reordered'] = data_train['reordered'].fillna(0)
data_train.head(5)

,user_id,product_id,ipu_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,last_order_size,items_total_purchases,items_reorder_ratio,eval_set,order_id,reordered
0,1,196,10,1.000000,5.0,10,0.694915,9,35791,0.776480,train,1187899,1.0
1,1,10258,9,1.000000,5.0,10,0.694915,9,1946,0.713772,train,1187899,1.0
2,1,10326,1,0.166667,0.0,10,0.694915,9,5526,0.652009,train,1187899,0.0
3,1,12427,10,1.000000,5.0,10,0.694915,9,6476,0.740735,train,1187899,0.0
4,1,13032,3,0.333333,2.0,10,0.694915,9,3751,0.657158,train,1187899,1.0


In [43]:
data_train = data_train.drop(['eval_set', 'order_id'], axis=1)
data_train.head(5)

,user_id,product_id,ipu_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,last_order_size,items_total_purchases,items_reorder_ratio,reordered
0,1,196,10,1.000000,5.0,10,0.694915,9,35791,0.776480,1.0
1,1,10258,9,1.000000,5.0,10,0.694915,9,1946,0.713772,1.0
2,1,10326,1,0.166667,0.0,10,0.694915,9,5526,0.652009,0.0
3,1,12427,10,1.000000,5.0,10,0.694915,9,6476,0.740735,0.0
4,1,13032,3,0.333333,2.0,10,0.694915,9,3751,0.657158,1.0


In [44]:
data_test = data_df[data_df['eval_set'] == 'test']
data_test.head()

,user_id,product_id,ipu_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,last_order_size,items_total_purchases,items_reorder_ratio,eval_set,order_id
120,3,248,1,0.090909,0.0,12,0.625,6,6371,0.400251,test,2774568
121,3,1005,1,0.333333,1.0,12,0.625,6,463,0.440605,test,2774568
122,3,1819,3,0.333333,0.0,12,0.625,6,2424,0.492162,test,2774568
123,3,7503,1,0.100000,0.0,12,0.625,6,12474,0.553551,test,2774568
124,3,8021,1,0.090909,0.0,12,0.625,6,27864,0.591157,test,2774568


In [45]:
data_test = data_test.drop(['eval_set','order_id'], axis=1)

Сохраним датасеты в формате CSV для отдельной разработки модели

In [46]:
data_train.to_csv(data_path + 'train_val_dataset.csv', index=False)

In [47]:
data_test.to_csv(data_path + 'test_dataset.csv', index=False)